In [1]:
# import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression, Ridge # Linear Regression and Ridge models
from xgboost import XGBRegressor # XGBoost model

from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score)

# import parquet support libraries
import pyarrow
import fastparquet

In [2]:
# read in the dataset
train_data = pd.read_parquet("training.parquet")
train_data.head()

,HOUR_OF_ARRIVAL,DELAY_ARR,LINE_NO_ARR_0/1,LINE_NO_ARR_0/2,LINE_NO_ARR_0/3,LINE_NO_ARR_0/4,LINE_NO_ARR_0/5,LINE_NO_ARR_0/6,LINE_NO_ARR_112,LINE_NO_ARR_118,...,FINAL_STOP_LOCATION_OOSTENDE,FINAL_STOP_LOCATION_OTHER LOCATION,FINAL_STOP_LOCATION_OTTIGNIES,FINAL_STOP_LOCATION_POPERINGE,FINAL_STOP_LOCATION_QUIEVRAIN,FINAL_STOP_LOCATION_SCHAARBEEK,FINAL_STOP_LOCATION_SINT-NIKLAAS,FINAL_STOP_LOCATION_TOURNAI,FINAL_STOP_LOCATION_TURNHOUT,FINAL_STOP_LOCATION_ZOTTEGEM
0,15,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,17,-52.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,21,-84.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,19,91.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,11,-57.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# check data types of the columns
train_data.info()

# This is aggregated

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7607505 entries, 0 to 7607504
Columns: 173 entries, HOUR_OF_ARRIVAL to FINAL_STOP_LOCATION_ZOTTEGEM
dtypes: float64(1), int32(1), uint8(171)
memory usage: 1.3 GB


In [4]:
# check the data types of all columns
train_data.dtypes

HOUR_OF_ARRIVAL                       int32
DELAY_ARR                           float64
LINE_NO_ARR_0/1                       uint8
LINE_NO_ARR_0/2                       uint8
LINE_NO_ARR_0/3                       uint8
                                     ...   
FINAL_STOP_LOCATION_SCHAARBEEK        uint8
FINAL_STOP_LOCATION_SINT-NIKLAAS      uint8
FINAL_STOP_LOCATION_TOURNAI           uint8
FINAL_STOP_LOCATION_TURNHOUT          uint8
FINAL_STOP_LOCATION_ZOTTEGEM          uint8
Length: 173, dtype: object

In [5]:
# convert columns with uint data type to integer
int_columns = train_data.columns.drop(["HOUR_OF_ARRIVAL", "DELAY_ARR"]).tolist()

train_data[int_columns] = train_data[int_columns].astype(int)

# check the new data types
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7607505 entries, 0 to 7607504
Columns: 173 entries, HOUR_OF_ARRIVAL to FINAL_STOP_LOCATION_ZOTTEGEM
dtypes: float64(1), int32(172)
memory usage: 4.9 GB


In [6]:
# Check if there are any missing values
train_data.isnull().sum()

HOUR_OF_ARRIVAL                     0
DELAY_ARR                           0
LINE_NO_ARR_0/1                     0
LINE_NO_ARR_0/2                     0
LINE_NO_ARR_0/3                     0
                                   ..
FINAL_STOP_LOCATION_SCHAARBEEK      0
FINAL_STOP_LOCATION_SINT-NIKLAAS    0
FINAL_STOP_LOCATION_TOURNAI         0
FINAL_STOP_LOCATION_TURNHOUT        0
FINAL_STOP_LOCATION_ZOTTEGEM        0
Length: 173, dtype: int64

In [7]:
# Find the correlation matrix
corr_matrix = train_data.corr()
corr_matrix

,HOUR_OF_ARRIVAL,DELAY_ARR,LINE_NO_ARR_0/1,LINE_NO_ARR_0/2,LINE_NO_ARR_0/3,LINE_NO_ARR_0/4,LINE_NO_ARR_0/5,LINE_NO_ARR_0/6,LINE_NO_ARR_112,LINE_NO_ARR_118,...,FINAL_STOP_LOCATION_OOSTENDE,FINAL_STOP_LOCATION_OTHER LOCATION,FINAL_STOP_LOCATION_OTTIGNIES,FINAL_STOP_LOCATION_POPERINGE,FINAL_STOP_LOCATION_QUIEVRAIN,FINAL_STOP_LOCATION_SCHAARBEEK,FINAL_STOP_LOCATION_SINT-NIKLAAS,FINAL_STOP_LOCATION_TOURNAI,FINAL_STOP_LOCATION_TURNHOUT,FINAL_STOP_LOCATION_ZOTTEGEM
HOUR_OF_ARRIVAL,1.000000,0.052601,-0.004691,0.008131,0.001805,0.003033,-0.000748,-0.002071,-0.002754,-0.001347,...,-0.005420,-0.001447,0.005828,-0.000279,0.008439,-0.004673,0.006627,0.007197,0.007723,-0.011032
DELAY_ARR,0.052601,1.000000,0.021546,0.016118,0.004500,0.009558,0.005626,0.001894,-0.001344,-0.004819,...,-0.005999,-0.009310,-0.019879,-0.007617,0.017227,-0.018515,0.002643,0.002827,-0.002270,0.005893
LINE_NO_ARR_0/1,-0.004691,0.021546,1.000000,-0.012096,-0.012750,-0.012439,-0.011274,-0.010519,-0.010776,-0.010920,...,-0.021186,0.015280,-0.013178,-0.011182,-0.011806,-0.003864,0.062818,-0.013694,-0.011221,-0.013082
LINE_NO_ARR_0/2,0.008131,0.016118,-0.012096,1.000000,-0.012539,-0.012233,-0.011088,-0.010345,-0.010597,-0.010739,...,0.054382,0.000226,-0.013050,-0.010996,-0.009849,-0.013216,-0.012189,-0.012635,-0.011047,0.061169
LINE_NO_ARR_0/3,0.001805,0.004500,-0.012750,-0.012539,1.000000,-0.012894,-0.011687,-0.010904,-0.011170,-0.011320,...,-0.021919,-0.029099,0.074898,-0.011591,-0.012238,0.053035,-0.011735,-0.014186,-0.010847,-0.013532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FINAL_STOP_LOCATION_SCHAARBEEK,-0.004673,-0.018515,-0.003864,-0.013216,0.053035,-0.013591,0.110817,-0.011493,0.000428,-0.011931,...,-0.023183,-0.057509,-0.014499,-0.012217,-0.012899,1.000000,-0.013542,-0.014962,-0.012273,-0.014305
FINAL_STOP_LOCATION_SINT-NIKLAAS,0.006627,0.002643,0.062818,-0.012189,-0.011735,-0.012534,-0.011313,-0.010599,-0.010858,-0.011004,...,-0.021380,-0.053038,-0.013371,-0.011267,-0.011896,-0.013542,1.000000,-0.013799,-0.011319,-0.013193
FINAL_STOP_LOCATION_TOURNAI,0.007197,0.002827,-0.013694,-0.012635,-0.014186,0.058348,-0.012499,0.019574,0.079742,0.115938,...,-0.023623,-0.058602,-0.014774,-0.012449,-0.013144,-0.014962,-0.013799,1.000000,-0.012506,-0.014576
FINAL_STOP_LOCATION_TURNHOUT,0.007723,-0.002270,-0.011221,-0.011047,-0.010847,-0.011360,0.075070,-0.009606,0.027674,-0.009973,...,-0.019377,-0.048069,-0.012119,-0.010211,-0.010782,-0.012273,-0.011319,-0.012506,1.000000,-0.011956


In [8]:
# Feature Selection
#corr_matrix = train_data.corr()
selected_features = corr_matrix[abs(corr_matrix['DELAY_ARR']) >= 0.01]['DELAY_ARR'].index.tolist()
train_data_subset = train_data[selected_features]

# check first few rows
train_data_subset

,HOUR_OF_ARRIVAL,DELAY_ARR,LINE_NO_ARR_0/1,LINE_NO_ARR_0/2,LINE_NO_ARR_124,LINE_NO_ARR_139,LINE_NO_ARR_161,LINE_NO_ARR_162,LINE_NO_ARR_25,LINE_NO_ARR_26,...,FINAL_STOP_LOCATION_GENT-SINT-PIETERS,FINAL_STOP_LOCATION_KORTRIJK,FINAL_STOP_LOCATION_LEUVEN,FINAL_STOP_LOCATION_LIEGE-GUILLEMINS,FINAL_STOP_LOCATION_LUXEMBOURG,FINAL_STOP_LOCATION_MECHELEN,FINAL_STOP_LOCATION_NOORDERKEMPEN,FINAL_STOP_LOCATION_OTTIGNIES,FINAL_STOP_LOCATION_QUIEVRAIN,FINAL_STOP_LOCATION_SCHAARBEEK
0,15,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,17,-52.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21,-84.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19,91.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,-57.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607500,17,15.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7607501,14,-34.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7607502,16,-5.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7607503,19,-47.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# subset the dependent features
X_train = train_data_subset.drop('DELAY_ARR', axis=1)

# subset the target feature
y_train = train_data_subset['DELAY_ARR']

In [10]:
# display dependent features
X_train

,HOUR_OF_ARRIVAL,LINE_NO_ARR_0/1,LINE_NO_ARR_0/2,LINE_NO_ARR_124,LINE_NO_ARR_139,LINE_NO_ARR_161,LINE_NO_ARR_162,LINE_NO_ARR_25,LINE_NO_ARR_26,LINE_NO_ARR_35,...,FINAL_STOP_LOCATION_GENT-SINT-PIETERS,FINAL_STOP_LOCATION_KORTRIJK,FINAL_STOP_LOCATION_LEUVEN,FINAL_STOP_LOCATION_LIEGE-GUILLEMINS,FINAL_STOP_LOCATION_LUXEMBOURG,FINAL_STOP_LOCATION_MECHELEN,FINAL_STOP_LOCATION_NOORDERKEMPEN,FINAL_STOP_LOCATION_OTTIGNIES,FINAL_STOP_LOCATION_QUIEVRAIN,FINAL_STOP_LOCATION_SCHAARBEEK
0,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607500,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7607501,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7607502,16,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7607503,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# display target features
y_train

0           3.0
1         -52.0
2         -84.0
3          91.0
4         -57.0
           ... 
7607500    15.0
7607501   -34.0
7607502    -5.0
7607503   -47.0
7607504    10.0
Name: DELAY_ARR, Length: 7607505, dtype: float64

In [12]:
# load the test data
test_data = pd.read_parquet("test.parquet")

In [13]:
# subset data for the selected columns only
test_data_subset = test_data[selected_features]

# subset the dependent and target features
X_test = test_data_subset.drop('DELAY_ARR', axis=1)
y_test = test_data_subset['DELAY_ARR']

In [14]:
# convert negative values to zero
def convertToZero(num):
     return 0 if num < 0 else num

In [15]:
# apply the function to y_train to convert negative values to 0
y_train = y_train.apply(convertToZero)

# check the values of y_train again
y_train

0           3.0
1           0.0
2           0.0
3          91.0
4           0.0
           ... 
7607500    15.0
7607501     0.0
7607502     0.0
7607503     0.0
7607504    10.0
Name: DELAY_ARR, Length: 7607505, dtype: float64

In [16]:
# apply the function to y_train to convert negative values to 0
y_test = y_test.apply(convertToZero)

# check the values of y_train again
y_test

7607505      0.0
7607506    133.0
7607507    162.0
7607508      0.0
7607509      0.0
           ...  
9509377    107.0
9509378    167.0
9509379     19.0
9509380     12.0
9509381      0.0
Name: DELAY_ARR, Length: 1901877, dtype: float64

### MODELLING

In [17]:
# Build model
LRmodel = LinearRegression()

# Fit the Model
LRmodel.fit(X_train, y_train)

LinearRegression()

In [18]:
# make predictions on the test data using the model
y_LR_pred = LRmodel.predict(X_test)

In [19]:
# calculate the metrics of the model
LRmse = mean_squared_error(y_test, y_LR_pred)
LRr2 = r2_score(y_test, y_LR_pred)

print(f"The MSE is: ${LRmse}.")
print(f"The R2 is: ${LRr2}.")

The MSE is: $116654.32103162873.
The R2 is: $0.028924634470745447.


In [20]:
# Find the accuracy of the model
accuracy = LRmodel.score(X_test, y_test)
accuracy

0.028924634470745447

#### 2. Extreme Gradient Boosting Model

In [21]:
# Build XGB model
XGBmodel = XGBRegressor(n_estimators=10, learning_rate=0.05, n_jobs=-1)
XGBmodel.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=-1, num_parallel_tree=None, predictor=None,
             random_state=None, ...)

In [22]:
# make predictions using the XGB model
y_XGB_pred = XGBmodel.predict(X_test)

In [23]:
# evaluate the performance of the model
XGBmse = mean_squared_error(y_test, y_XGB_pred)
XGBr2 = r2_score(y_test, y_XGB_pred)

print("XGB MSE:", XGBmse)
print("XGB R2:", XGBr2)

XGB MSE: 124823.0154959373
XGB R2: -0.039074715169072016


In [59]:
# Fine tune the model
param_grid = {
    'n_estimators': [10, 20, 30, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(XGBmodel, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)


# This is causing my laptop to hang

MemoryError: Unable to allocate 412. MiB for an array with shape (71, 6086004) and data type uint8

#### 3. Ridge Regression

In [24]:
# Train the model with grid search
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10]})

In [25]:
# Select the best model
Ridgemodel = grid_search.best_estimator_

In [26]:
# Evaluate the model
y_Ridge_pred = Ridgemodel.predict(X_test)
Ridgemse = mean_squared_error(y_test, y_Ridge_pred)
Ridger2 = r2_score(y_test, y_Ridge_pred)

print("MSE:", Ridgemse)
print("R^2:", Ridger2)

MSE: 116654.32522756369
R^2: 0.028924599542171525
